In [46]:
"""
Created on Fri Feb 19 08:46:16 2021

@author: Schroeder
"""

# Liest holc etc. daten von bereits auf platte befindlichen csv dataframes .
# Soll Plot und Auswertung von datagrabbing trennen.

import plotly.io as pio
import plotly.express as px
from plotly.figure_factory import create_candlestick
import plotly.graph_objects as go
from plotly.offline import init_notebook_mode, iplot

import TickerSelector, regression
from datetime import datetime
from datetime import date
import calendar

# Imports
from pandas_datareader import data as pdr
#from yahoo_fin import stock_info as si

import numpy as np
import scipy.stats as stats


from pandas import ExcelWriter
import yfinance as yf
import pandas as pd
import datetime
import time
import csv
   
import StockscreenerPlotter_stats
import StockscreenerWinners_stats
import downloader

import os.path
from os.path import isfile, join

from os import listdir
#from os.path import isfile, join

from lib import Indikatoren

#lieferrt bei gegebenem datatfram mit datum als index das erste Datum, das letzte, und zwei 
# dazwischen


    

## Gebe Pfad an, unter dem das Datenfile zu finden ist.
## Bsp: C:/Users/Schroeder/OneDrive/Trading/ETFS/RES_Aristokraten/Res/


# --------------------------------------------------------------------------
## Welcher Sektor oder index soll analysieert werden ?

In [47]:
pfad = "C:/Users/Schroeder/OneDrive/Trading/ETFS/"
#############################################################
who = "RES_SP500"

###############################################################
pfad = pfad+who+"/Data/"
file  = "_ticker_names.csv"

singleticker_file=pfad + file



In [51]:
exportList = pd.DataFrame(columns=['ticker',"z-score","MACDsignal","RSIlevel","Name","Industry","Sector","marketCap"])

In [52]:
mydict={}

## ---------------------------------------------------------------------

In [54]:
# DAs ist der Ticker fiel, der alle Ticker enthaelt, die gelsen werden:
tkr = pd.read_csv(singleticker_file,sep=";",decimal=',',parse_dates=True,index_col=0)
        #df = yf.download(ticker, start=start_date.strftime("%Y-%m-%d"), end=end_date.strftime("%Y-%m-%d"))
    
for i in tkr["ticker"]:
    file=pfad+i +".csv"
    df = pd.read_csv(file,sep=";",decimal=',',parse_dates=True,index_col=0)
    #print(file)
    df["dummy"]= df["Factor"]-df["EMA100"]
    df["dummy"]=df["dummy"].fillna(0)
    Indikatoren.Zscore_rolling(df,"dummy",100)
    #df["Zscore"]=stats.zscore(df["dummy"])
    #Prüfe auf z-score
    if (df["z-score"][-1] < -1.5) or (df["z-score"][-1] > 1.5):
        mydict["ticker"]=i
        mydict["z-score"]=df["z-score"][-1]
        mydict["Name"]=tkr["name"][tkr["ticker"]==i]
        
    if((df["MACDdif_week"][-1] > 0) and \
        ((df["MACD_week"][-1] > 0) and (df["MACDsign_week"][-1] < 0))):
            mydict["MACDsignal"]="Long"
    if((df["MACDdif_week"][-1] < 0) and \
        ((df["MACD_week"][-1] < 0) and (df["MACDsign_week"][-1] > 0))):
            mydict["MACDsignal"]="Short"        
    exportList =exportList.append(mydict,ignore_index=True) 
    
    if i == "HRB":
        fig = px.line(df, x=df.index, y=[df["MACD_week"],df["MACDsign_week"]])
        fig.show()   
#exportList.style.background_gradient()
print(exportList.sort_values(by="z-score",ascending=False))


    ticker   z-score MACDsignal  RSIlevel  \
392   SBUX  2.306218      Short       NaN   
389   SBUX  2.306218      Short       NaN   
870   SBUX  2.306218      Short       NaN   
869   SBUX  2.306218      Short       NaN   
868   SBUX  2.306218      Short       NaN   
..     ...       ...        ...       ...   
6      NaN       NaN        NaN       NaN   
7      NaN       NaN        NaN       NaN   
8      NaN       NaN        NaN       NaN   
9      NaN       NaN        NaN       NaN   
10     NaN       NaN        NaN       NaN   

                                                  Name  Industry  Sector  \
392  402    Starbucks Corporation
Name: name, dtype...       NaN     NaN   
389  402    Starbucks Corporation
Name: name, dtype...       NaN     NaN   
870  402    Starbucks Corporation
Name: name, dtype...       NaN     NaN   
869  402    Starbucks Corporation
Name: name, dtype...       NaN     NaN   
868  402    Starbucks Corporation
Name: name, dtype...       NaN     NaN   
.. 